In [ ]:
import os
import sys
import torch
import torchvision

os.chdir("..")

%load_ext autoreload
%autoreload 2

from detectron2.utils import comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import (
    DefaultTrainer,
    DefaultPredictor,
    default_argument_parser,
    default_setup,
    launch,
)
from detectron2.evaluation import COCOEvaluator, verify_results
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
import random, cv2
import numpy as np
# import albumentations as A

sys.path.append("detectron2_repo/projects/TensorMask")

from lib import coco_handler, copy_and_paste_augm, constants
from lib.detectron2_utils import *
from lib.copy_and_paste_detectron import *

from tensormask import add_tensormask_config
import tensormask

import warnings
from glob import glob

In [ ]:
cfg = get_cfg()
add_tensormask_config(cfg)
cfg.merge_from_file("configs/fixed_image_config.yaml")
# cfg.DATASETS.TRAIN = ("train",)
# cfg.DATASETS.TEST = ("val",)
# cfg.OUTPUT_DIR = "./output/better_aug_empty_images/"
# cfg.DATALOADER.NUM_WORKERS = 2
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.SOLVER.MAX_ITER = 30001
# cfg.SOLVER.STEPS = (20000, 30000)
cfg.MODEL.TENSOR_MASK.NUM_CLASSES = 1
# cfg.TEST.EVAL_PERIOD = 500
# cfg.SOLVER.LR = 0.0005
# cfg.MODEL.TENSOR_MASK.SCORE_THRESH_TEST = 0.25
create_output(cfg)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join("output", "Colab_train", "model_final.pth")
cfg.MODEL.TENSOR_MASK.SCORE_THRESH_TEST = 0.10
predictor = DefaultPredictor(cfg)

NOTE: stuff seems to work quite bad if the images are processed with out histogram spreading (preview option during processing). To archive reasonable performance use this option during image extraction!!

In [ ]:
fails = []
for s in glob("../data/annotated/*/"):
    dis = s.split("/")[-2]
    if "bone_age" in dis:
        continue
    os.makedirs(f"output_new/{dis}", exist_ok=True)
    print(dis)
    for img_path in tqdm(glob(f"../data/annotated/{dis}/*")):
        try:
            im = cv2.imread(img_path).astype(np.float32)
            im -= np.min(im)
            im /= np.max(im)
            im = (im * 255).astype(np.uint8)
            outputs = predictor(im)
            masks = outputs["instances"].pred_masks.cpu().numpy().astype(np.uint8)
            n = masks.shape[0]
            conts = []
            for i in range(n):
                contours , _ = cv2.findContours(masks[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                conts += contours
            c = max(conts, key = cv2.contourArea)
            pred = cv2.drawContours(np.zeros(masks[i].shape), [c], -1, 255, -1)
            pred *= 255
            cv2.imwrite(f"output_new/{dis}/{os.path.basename(img_path)}", pred)
        except:
            print(f"no mask found in {img_path}")
            fails.append(img_path)
        break

In [ ]:
fails = []
for s in glob("../data/annotated/*/"):
    dis = s.split("/")[-2]
    if "bone_age" in dis:
        continue
    os.makedirs(f"output_new/{dis}", exist_ok=True)
    print(dis)
    for img_path in glob(f"../data/annotated/{dis}/*"):
        try:
            im = cv2.imread(img_path).astype(np.float32)
#             im -= np.min(im)
#             im /= np.max(im)
#             im = (im * 255).astype(np.uint8)
            outputs = predictor(im)
            masks = outputs["instances"].pred_masks.cpu().numpy().astype(np.uint8)
            n = masks.shape[0]
            conts = []
            for i in range(n):
                contours , _ = cv2.findContours(masks[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                conts += contours
            c = max(conts, key = cv2.contourArea)
            pred = cv2.drawContours(np.zeros(masks[i].shape), [c], -1, 255, -1)
            pred *= 255
            cv2.imwrite(f"output_new/{dis}/{os.path.basename(img_path)}", pred)
        except:
            print(f"no mask found in {img_path}")
            fails.append(img_path)

In [ ]:
for s in tqdm(glob("../data/annotated/rsna_bone_age/bone_age_training_data_set/*")):
    n = int(s.split("/")[-1].replace(".png", ""))
    if not os.path.exists(f"output/Colab_train/inference/bone_age_train/{n}.png"):
        try:
            im = cv2.imread(s).astype(np.float32)
            im -= np.min(im)
            im /= np.max(im)
            im = (im * 255).astype(np.uint8)
            outputs = predictor(im)
            pred = outputs["instances"].pred_masks.cpu().numpy()[0, :, :].astype(np.uint8)
            pred *= 255
            pred = pred
            cv2.imwrite(f"output/Colab_train/inference/bone_age_train/{n}.png", pred)
        except:
            print(f"no mask found in {os.path.basename(s)}")